In [1]:
# # Debugging ONLY! autoreloads when scripts update
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import contextily as ctx

import os

from src import preprocessing, config

# Obtain raw data

In [2]:
# Get 2020 Decennial Census data
decennial2020_dp_raw = preprocessing.fetch_2020_demographic_profile()

GET request status: 200


In [3]:
np.array(decennial2020_dp_raw).shape

(2328, 646)

# Perform initial cleaning

In [3]:
# Clean NYC census tract shapefiles
gdf = preprocessing.clean_tracts(config.TRACTS_RAW, config.TRACTS_CLEAN)

gdf.sample(n=5)

,BOROUGH,TRACT,AREA,LAT,LONG,geometry
GEOID,,,,,,
36005006200,The Bronx,62,194004,40.833288,-73.875519,"POLYGON ((-73.87925 40.83442, -73.87841 40.835..."
36047005602,Brooklyn,56.02,135056,40.612738,-74.035739,"POLYGON ((-74.04645 40.61073, -74.03903 40.613..."
36081073700,Queens,737,210609,40.718440,-73.843249,"POLYGON ((-73.84623 40.71787, -73.8459 40.7184..."
36047030800,Brooklyn,308,681579,40.586551,-73.979511,"POLYGON ((-73.98742 40.58513, -73.98729 40.585..."
36061014601,Manhattan,146.01,74344,40.777192,-73.952161,"POLYGON ((-73.95448 40.77814, -73.95398 40.778..."
